In [2]:
import tensorflow as tf

import numpy as np

import matplotlib.pyplot as plt

import os

import time

from tqdm.notebook import tqdm

In [3]:
cur_dir = os.getcwd()
model_dir_name = "minst_cnn_func"

ckpt_dir = os.path.join(cur_dir, "checkpoints", model_dir_name)
os.makedirs(ckpt_dir, exist_ok=True)

ckpt_prefix = os.path.join(ckpt_dir, model_dir_name)

In [4]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()    

train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.

train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)
    
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)    

batch_size = 100

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(len(train_images)).batch(batch_size, drop_remainder=True)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

In [6]:
def create_model():
    inputs = tf.keras.Input(shape=(28, 28, 1))
    
    conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu)(inputs)
    pool1 = tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding="same")(conv1)
    
    conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu)(pool1)
    pool2 = tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding="same")(conv2)
    
    conv3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu)(pool2)
    pool3 = tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding="same")(conv3)
    
    flatten1 = tf.keras.layers.Flatten()(pool3)
    
    dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)(flatten1)
    dropout1 = tf.keras.layers.Dropout(rate=0.4)(dense1)
    logits = tf.keras.layers.Dense(units=10)(dropout1)
    
    return tf.keras.Model(inputs=inputs, outputs=logits)

model = create_model()

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0     

In [8]:
@tf.function
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, from_logits=True))    
    return loss

@tf.function
def grad_fn(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)


@tf.function
def acc_fn(model, images, labels):
    logits = model(images, training=False)
    is_equal = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    acc = tf.reduce_mean(tf.cast(is_equal, tf.float32))
    return acc

In [9]:
lr = 0.001
training_epochs = 15

tf.random.set_seed(0)

ckpt = tf.train.Checkpoint(cnn=model)

start_time = time.time()
for epoch in tqdm(range(training_epochs)):
    avg_train_loss = 0.
    avg_train_acc = 0.

    avg_test_acc = 0.
    
    train_step = 0
    test_step = 0    
    
    for images, labels in tqdm(train_dataset):
        grads = grad_fn(model, images, labels)    
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        optimizer.apply_gradients(zip(grads, model.variables))
        loss = loss_fn(model, images, labels)
        acc = acc_fn(model, images, labels)
        avg_train_loss += loss
        avg_train_acc += acc
        train_step += 1
        
    avg_train_loss = avg_train_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in tqdm(test_dataset):
        acc = acc_fn(model, images, labels)        
        avg_test_acc += acc
        test_step += 1
    
    avg_test_acc = avg_test_acc / test_step    

    print("epoch : {} | loss : {:.6f}| train acc : {:.4f}| test acc :{:.4f}".format(epoch + 1, avg_train_loss, avg_train_acc, avg_test_acc))
    
    ckpt.save(file_prefix=ckpt_prefix)

print("elapsed time :", time.time() - start_time)


epoch : 1 | loss : 0.173349| train acc : 0.9494| test acc :0.9773



epoch : 2 | loss : 0.020948| train acc : 0.9933| test acc :0.9839



epoch : 3 | loss : 0.016052| train acc : 0.9945| test acc :0.9858



epoch : 4 | loss : 0.018900| train acc : 0.9941| test acc :0.9821



epoch : 5 | loss : 0.017942| train acc : 0.9949| test acc :0.9803


...


epoch : 11 | loss : 0.014275| train acc : 0.9964| test acc :0.9896



epoch : 12 | loss : 0.012006| train acc : 0.9970| test acc :0.9883



epoch : 13 | loss : 0.008722| train acc : 0.9977| test acc :0.9891



epoch : 14 | loss : 0.006794| train acc : 0.9978| test acc :0.9918



epoch : 15 | loss : 0.007509| train acc : 0.9980| test acc :0.9884

elapsed time : 2565.4364714622498
